# Stage 3

There are 3 main sections to this notebook. The first section deals with creating a meta-dataframe which is useful for general visualistaion of the data. The second section deals with expanding and labeling everyrun.  The third section? deals with the extraction of a relative noise profile from the raw counts. The final section deals with creating a dataframe where the noise profile is extracted from every run and is labeled. 

1. [The Meta-Dataframe](#1-the-meta-dataframe)
2. [The Relative Noise Profile](#2-the-relative-noise-profile)
3. [A Fully Labeled Dataframe](#a-fully-labeled-dataframe)

### Imports

In [2]:
import sys
sys.path.append('../')

from investigation_functions import  meta_dataframe_functions as mdf
from investigation_functions import  data_process_funcs as dpf
import config
import backend_vars

import pandas as pd

The relative directory of the different experimnet type folders must be set correctly:

In [3]:
dir = "../../"

## 1. The meta-dataframe

The combination blank_meta_df() and load_meta_df() produce a pandas dataframe from the csv files that contain the raw counts. The dataframe contains the following labels:
- nr_qubits : the number of qubits of the circuit (4, 8, 16)
- backend : the backend that the circuit was run on (brisbane, fez, marrakesh, torino)
- sim	: whether the backend was simulated (True) or not (False)
- circuit_type	: the type of circuit that was run (1, 2, 3)
- file_path : the filepath of the csv file containing the raw counts

The type of experiment, 'Hardware','Simulation', and 'Refreshed_Simulation' must be specified to load the results from the folder. This 'meta-dataframe' is useful for sorting the data before loading it all. Loading the data can be a lenghty process.

In [4]:
df_Refr_Sim = mdf.blank_meta_df()
df_Sim =mdf.blank_meta_df()
df_Hardware = mdf.blank_meta_df()

mdf.load_meta_df(df_Refr_Sim,'Refreshed_Simulation',dir)
mdf.load_meta_df(df_Sim,'Simulation',dir)
mdf.load_meta_df(df_Hardware,'Hardware',dir)

df_Refr_Sim.tail()

,nr_qubits,backend,sim,circuit_type,file_path
13,16,brisbane,True,2,../../Refreshed_Simulated_results/16q/16q_fake...
14,16,brisbane,True,3,../../Refreshed_Simulated_results/16q/16q_fake...
15,16,torino,True,1,../../Refreshed_Simulated_results/16q/16q_fake...
16,16,torino,True,2,../../Refreshed_Simulated_results/16q/16q_fake...
17,16,torino,True,3,../../Refreshed_Simulated_results/16q/16q_fake...


Various columns can be added to this 'meta dataframe'. Such as 'experiment' type, a dataframe containing the raw counts, and a measure of the sparsity of the counts in that dataframe. This can be useful when trying to understand the data further. See [Visualising the Sparsity](../sparse_stuff.ipynb).

In [5]:
df_Refr_Sim =mdf.add_experiment_type_column(df_Refr_Sim)
df_Refr_Sim.head()

,nr_qubits,backend,sim,circuit_type,file_path,experiment_type
0,4,brisbane,True,1,../../Refreshed_Simulated_results/4q/4q_fake_b...,Refreshed Sim
1,4,brisbane,True,2,../../Refreshed_Simulated_results/4q/4q_fake_b...,Refreshed Sim
2,4,brisbane,True,3,../../Refreshed_Simulated_results/4q/4q_fake_b...,Refreshed Sim
3,4,torino,True,1,../../Refreshed_Simulated_results/4q/4q_fake_t...,Refreshed Sim
4,4,torino,True,2,../../Refreshed_Simulated_results/4q/4q_fake_t...,Refreshed Sim


## 2. The Relative Noise Profile

First, a single csv file is processed. The total erroneous counts are calculated and added to the Dataframe of raw couts in the column 'totalError'.

In [6]:
unprocessed_df = dpf.create_unprocessed_df(df_Refr_Sim.loc[0,'file_path'])
unprocessed_df.head()

,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,143,143,66,38,0.0,29,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,132,132,60,43,0.0,25,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,127,127,46,41,0.0,30,1.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,128,128,60,32,1.0,28,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,116,116,48,31,0.0,30,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The erroneus counts are then divided by the totalError to get the percentage distribution of the error over the incorrect outcomes.

In [7]:
processed_df = dpf.create_processed_df(df_Refr_Sim.loc[0,'file_path'])
processed_df.head()

,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,143,1.0,0.461538,0.265734,0.000000,0.202797,0.000000,0.000000,0.0,0.069930,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,132,1.0,0.454545,0.325758,0.000000,0.189394,0.000000,0.000000,0.0,0.030303,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,127,1.0,0.362205,0.322835,0.000000,0.236220,0.007874,0.007874,0.0,0.062992,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,128,1.0,0.468750,0.250000,0.007812,0.218750,0.000000,0.007812,0.0,0.039062,0.007812,0.0,0.0,0.0,0.0,0.0,0.0
4,116,1.0,0.413793,0.267241,0.000000,0.258621,0.000000,0.008621,0.0,0.051724,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


## A Fully-Labeled Dataframe

First, the meta-dataframe is created according to the experiment type requested, then the rows that correpsond the the requested number of qubits are expanded, processed, and returned.

In [8]:
nr_qubits = 4
df_arr_R = dpf.get_expanded_df('Refreshed_Simulation',nr_qubits,dir)

In [9]:
df_arr_R.head()

,circuit_type,backend,nr_qubits,experiment_type,totalError,0000,0001,0010,0011,0100,...,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,brisbane,4,Refreshed_Simulation,143,1.0,0.461538,0.265734,0.000000,0.202797,...,0.000000,0.0,0.069930,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,brisbane,4,Refreshed_Simulation,132,1.0,0.454545,0.325758,0.000000,0.189394,...,0.000000,0.0,0.030303,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,1,brisbane,4,Refreshed_Simulation,127,1.0,0.362205,0.322835,0.000000,0.236220,...,0.007874,0.0,0.062992,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,1,brisbane,4,Refreshed_Simulation,128,1.0,0.468750,0.250000,0.007812,0.218750,...,0.007812,0.0,0.039062,0.007812,0.0,0.0,0.0,0.0,0.0,0.0
4,1,brisbane,4,Refreshed_Simulation,116,1.0,0.413793,0.267241,0.000000,0.258621,...,0.008621,0.0,0.051724,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


## Next stage -> [Stage 4 ](Stage4_Preprocessing_Data.ipynb)